---
title: "Multinomial Logit Model"
author: "Nicole Ziola"
date: 05-25-2025
---


This assignment expores two methods for estimating the MNL model: (1) via Maximum Likelihood, and (2) via a Bayesian approach using a Metropolis-Hastings MCMC algorithm. 


## 1. Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 3 products, then either $y=3$ or $y=(0,0,1)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, brand, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 3 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^3e^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$



## 2. Simulate Conjoint Data

We will simulate data from a conjoint experiment about video content streaming services. We elect to simulate 100 respondents, each completing 10 choice tasks, where they choose from three alternatives per task. For simplicity, there is not a "no choice" option; each simulated respondent must select one of the 3 alternatives. 

Each alternative is a hypothetical streaming offer consistent of three attributes: (1) brand is either Netflix, Amazon Prime, or Hulu; (2) ads can either be part of the experience, or it can be ad-free, and (3) price per month ranges from \$4 to \$32 in increments of \$4.

The part-worths (ie, preference weights or beta parameters) for the attribute levels will be 1.0 for Netflix, 0.5 for Amazon Prime (with 0 for Hulu as the reference brand); -0.8 for included adverstisements (0 for ad-free); and -0.1*price so that utility to consumer $i$ for hypothethical streaming service $j$ is 

$$
u_{ij} = (1 \times Netflix_j) + (0.5 \times Prime_j) + (-0.8*Ads_j) - 0.1\times Price_j + \varepsilon_{ij}
$$

where the variables are binary indicators and $\varepsilon$ is Type 1 Extreme Value (ie, Gumble) distributed.

The following code provides the simulation of the conjoint data.

:::: {.callout-note collapse="true"}

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(123)

brand = ["N", "P", "H"] 
ad = ["Yes", "No"]
price = np.arange(8, 33, 4)

import itertools
profiles = pd.DataFrame(list(itertools.product(brand, ad, price)), columns=["brand", "ad", "price"])
m = len(profiles)

b_util = {"N": 1.0, "P": 0.5, "H": 0.0}
a_util = {"Yes": -0.8, "No": 0.0}
p_util = lambda p: -0.1 * p

n_peeps = 100
n_tasks = 10
n_alts = 3

def sim_one(id):
    datlist = []

    for t in range(1, n_tasks + 1):
        sampled = profiles.sample(n=n_alts).copy()
        sampled.insert(0, "resp", id)
        sampled.insert(1, "task", t)

        sampled["v"] = (
            sampled["brand"].map(b_util) +
            sampled["ad"].map(a_util) +
            sampled["price"].apply(p_util)
        ).round(10)

        sampled["e"] = -np.log(-np.log(np.random.rand(n_alts)))
        sampled["u"] = sampled["v"] + sampled["e"]

        sampled["choice"] = (sampled["u"] == sampled["u"].max()).astype(int)

        datlist.append(sampled)

    return pd.concat(datlist, ignore_index=True)

conjoint_data = pd.concat([sim_one(i) for i in range(1, n_peeps + 1)], ignore_index=True)

conjoint_data = conjoint_data[["resp", "task", "brand", "ad", "price", "choice"]]

print(conjoint_data.head())

## 3. Preparing the Data for Estimation

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). The fact that each task for each respondent has the same number of alternatives (3) helps.  In addition, we need to convert the categorical variables for brand and ads into binary variables.

In [ ]:
import pandas as pd

df = pd.read_csv('conjoint_data.csv')

import pandas as pd

conjoint_clean = pd.get_dummies(df, columns=["brand", "ad"], drop_first=True)

conjoint_clean = conjoint_clean.rename(columns={
    "brand_N": "brand_netflix",
    "brand_P": "brand_prime",
    "ad_Yes": "ad_yes"
})

conjoint_clean["alt_id"] = conjoint_clean.groupby(["resp", "task"]).cumcount() + 1

conjoint_clean = conjoint_clean[[
    "resp", "task", "alt_id",
    "brand_netflix", "brand_prime", "ad_yes", "price",
    "choice"
]]


print(conjoint_clean.head(10))

## 4. Estimation via Maximum Likelihood

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv

X = conjoint_clean[["brand_netflix", "brand_prime", "ad_yes", "price"]].astype(float).values
y = conjoint_clean["choice"].values

groups = conjoint_clean.groupby(["resp", "task"]).indices
choice_sets = list(groups.values())

def log_likelihood(beta):
    log_lik = 0
    for idx in choice_sets:
        X_set = X[idx]
        y_set = y[idx]
        utilities = X_set @ beta
        exp_utilities = np.exp(utilities)
        probs = exp_utilities / np.sum(exp_utilities)
        log_lik += np.log(probs[y_set == 1][0])
    return -log_lik  

initial_beta = np.zeros(X.shape[1])
result = minimize(log_likelihood, initial_beta, method="BFGS")

beta_hat = result.x
hessian_inv = result.hess_inv
std_errors = np.sqrt(np.diag(hessian_inv))

z = 1.96  # critical value for 95% CI
conf_ints = np.vstack([beta_hat - z * std_errors, beta_hat + z * std_errors]).T

param_names = ["beta_netflix", "beta_prime", "beta_ads", "beta_price"]
summary = pd.DataFrame({
    "Parameter": param_names,
    "Estimate": beta_hat,
    "Std. Error": std_errors,
    "95% CI Lower": conf_ints[:, 0],
    "95% CI Upper": conf_ints[:, 1]
})

print(summary)

## 5. Estimation via Bayesian Methods

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import logsumexp

X = conjoint_clean[["brand_netflix", "brand_prime", "ad_yes", "price"]].astype(float).values
y = conjoint_clean["choice"].values
group_keys = conjoint_clean[["resp", "task"]].apply(tuple, axis=1)
choice_sets = [np.where(group_keys == key)[0] for key in sorted(set(group_keys))]

def log_likelihood(beta):
    log_lik = 0.0
    for idx in choice_sets:
        X_set = X[idx]
        y_set = y[idx]
        utilities = X_set @ beta
        log_lik += utilities[y_set == 1][0] - logsumexp(utilities)
    return log_lik


def log_prior(beta):
    prior_var = np.array([25, 25, 25, 1])  
    return -0.5 * np.sum((beta ** 2) / prior_var)

def log_posterior(beta):
    return log_likelihood(beta) + log_prior(beta)

n_draws = 11000
burn_in = 1000
n_params = 4
samples = np.zeros((n_draws, n_params))
beta_current = np.zeros(n_params)
log_post_current = log_posterior(beta_current)

proposal_sd = np.array([0.05, 0.05, 0.05, 0.005])

for t in range(n_draws):
    proposal = beta_current + np.random.normal(loc=0, scale=proposal_sd)
    log_post_proposal = log_posterior(proposal)
    
    log_accept_ratio = log_post_proposal - log_post_current
    if np.log(np.random.rand()) < log_accept_ratio:
        beta_current = proposal
        log_post_current = log_post_proposal
    
    samples[t] = beta_current

samples_post = samples[burn_in:]
posterior_means = np.mean(samples_post, axis=0)
posterior_std = np.std(samples_post, axis=0)
posterior_ci_lower = np.percentile(samples_post, 2.5, axis=0)
posterior_ci_upper = np.percentile(samples_post, 97.5, axis=0)

param_names = ["beta_netflix", "beta_prime", "beta_ads", "beta_price"]
posterior_summary = pd.DataFrame({
    "Parameter": param_names,
    "Posterior Mean": posterior_means,
    "Std. Dev": posterior_std,
    "95% CI Lower": posterior_ci_lower,
    "95% CI Upper": posterior_ci_upper
})

print(posterior_summary)

_todo: code up a metropolis-hasting MCMC sampler of the posterior distribution. Take 11,000 steps and throw away the first 1,000, retaining the subsequent 10,000._


In [ ]:
import matplotlib.pyplot as plt

posterior_samples = samples_post  

beta_idx = 0 
param_name = "Beta_Netflix"

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(posterior_samples[:, beta_idx], color="blue", alpha=0.6)
plt.title(f"Trace Plot: {param_name}")
plt.xlabel("Iteration")
plt.ylabel("Value")

plt.subplot(1, 2, 2)
plt.hist(posterior_samples[:, beta_idx], bins=30, color="skyblue", edgecolor="black")
plt.title(f"Posterior Distribution: {param_name}")
plt.xlabel("Value")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()

The results from the Maximum Likelihood approach are approximately the same compared to the estimation via Bayesian Methods. 



## 6. Discussion


If we did not simulate the data and only saw the parameter estimates, we would interpret the values as reflecting consumer preferences inferred from real-world consumer trends. $\beta_\text{Netflix} > \beta_\text{Prime}$ suggests that customers tend to prefer Netflix over Amazon Prime, which tracks with our intuition about streaming service brands. The higher beta parameter means that customers are more likely to choose Netflix over Prime, indicating Netflix's higher percieved value or brand equity. 

It makes sense that $\beta_\text{price}$ is negative because our intuition of supply and demand is that as prices increase, utility decrease. All else equal, consumers prefer lower costs. 

For "real world" conjoint data, we need to allow differences by individual. Therefore, we would need to allow the beta parameters to vary by individual, instead of having a fixed value across the population. Thus, the simulated data would need to have individual-level coefficients,generate choices based off of the individual beta parameters, and then estimate with a Bayesian hierarchical model.

This approach captures heterogeneity in preferences, which is a key aspect of real-world conjoint data, allowing different people to value different choices differently.
